In [1]:
import scipy.optimize as optimize
import numpy as np
import math

Numerical Procedures

In [2]:
class StockOption(object):
    def __init__(self, S0, K, rf, T, N, params):
        self.S0 = S0
        self.K = K
        self.T = T
        self.rf = rf
        self.N = max(1,N)
        self.STs = None
        
        self.pu = params.get("pu", 0) # probability of stock price moving up
        self.pd = params.get("pd", 0) # probability of stock price moving down
        self.div = params.get("div", 0) # dividend yield
        self.sigma = params.get("sigma", 0) # volatility
        self.is_call = params.get("is_call", True) # check call or put
        self.is_european = params.get("is_european", True) # check european vs american
        
        self.dt = T / float(N) # time step in years
        self.df = math.exp((rf-self.div)*self.dt) # discount factor        

In [3]:
class BinomialEuropeanOption(StockOption):
    
    def _setup_parameters_(self):
        self.M = self.N + 1
        self.u = 1 + self.pu
        self.d = 1 - self.pd
        self.qu = (math.exp((self.rf - self.div)*self.dt) - self.d) / (self.u - self.d)
        self.qd = 1 - self.qu
    
    def _initialize_stock_price_tree_(self):
        self.STs = np.zeros(self.M)
        
        for i in range(self.M):
            self.STs[i] = self.S0*(self.u**(self.N - i))*(self.d**i)
        
        a = self.STs
        return a
    
    def _initialize_payoffs_tree_(self):
        payoffs = np.maximum(0, (self.STs - self.K) if self.is_call else (self.K - self.STs))
        return payoffs
    
    def _traverse_tree_(self, payoffs):
        for i in range(self.N):
            payoffs = (payoffs[:-1]*self.qu + payoffs[1:]*self.qd) * self.df
        return payoffs
    
    def _begin_tree_traversal_(self):
        payoffs = self._initialize_payoffs_tree_()
        return self._traverse_tree_(payoffs)
    
    def price(self):
        self._setup_parameters_()
        self._initialize_stock_price_tree_()
        payoffs = self._begin_tree_traversal_()
        
        return payoffs[0]

In [4]:
eu_option = BinomialEuropeanOption(100, 100, 0.06, 1, 12,
        {"pu": 0.2, "pd": 0.2, "is_call": False})
print(eu_option.price())

27.195380234429283


In [5]:
class BinomialTreeOption(StockOption):
    
    def _setup_parameters_(self):
        self.M = self.N + 1
        self.u = 1 + self.pu
        self.d = 1 - self.pd
        self.qu = (math.exp((self.rf - self.div)*self.dt) - self.d) / (self.u - self.d)
        self.qd = 1 - self.qu
    
    def _initialize_stock_price_tree_(self):
        self.STs = [np.array([self.S0])]
        
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st  = np.concatenate((prev_branches*self.u, [prev_branches[-1]*self.d]))
            self.STs.append(st)
            
        return self.STs
    
    def _initialize_payoffs_tree_(self):
        payoffs = np.maximum(0, (self.STs[self.N] - self.K) if self.is_call else (self.K - self.STs[self.N]))
        return payoffs
    
    def _check_early_exercise_(self, payoffs, node):
        early_ex_payoff = (self.STs[node] - self.K) if self.is_call else (self.K - self.STs[node])
    
    def _traverse_tree_(self, payoffs):
        for i in reversed(range(self.N)):
            payoffs = (payoffs[:-1]*self.qu + payoffs[1:]*self.qd) * self.df
        
        if not self.is_european:
            payoffs = self._check_early_exercise_(payoffs, i)
        
        return payoffs
    
    def _begin_tree_traversal_(self):
        payoffs = self._initialize_payoffs_tree_()
        return self._traverse_tree_(payoffs)
    
    def price(self):
        self._setup_parameters_()
        self._initialize_stock_price_tree_()
        payoffs = self._begin_tree_traversal_()
        
        return payoffs[0]

In [6]:
am_option = BinomialTreeOption(
        50, 50, 0.05, 0.5, 2,
        {"pu": 0.2, "pd": 0.2, "is_call": False, "is_eu": False})

print(am_option.price())

5.073068207271768


In [7]:
class BinomialCRROption(BinomialTreeOption):
    def _setup_parameters_(self):
        self.u = math.exp(self.sigma * math.sqrt(self.dt))
        self.d = 1. / self.u
        self.qu = (math.exp((self.rf - self.div)*self.dt) - self.d) / (self.u - self.d)
        self.qd = 1 - self.qu

In [8]:
am_option = BinomialCRROption(
        50, 50, 0.05, 0.5, 2,
        {"sigma": 0.3, "is_call": False, "is_eu": False})
print("American put: %s" % am_option.price())

American put: 3.2643516498940786


In [9]:
len(am_option._begin_tree_traversal_())

1

In [10]:
class BinomialLROption(BinomialTreeOption):

    def _setup_parameters_(self):
        odd_N = self.N if (self.N%2 == 1) else (self.N+1)
        d1 = (math.log(self.S0/self.K) +  
              ((self.rf-self.div) +
               (self.sigma**2)/2.) *
              self.T) / (self.sigma * math.sqrt(self.T))
        d2 = (math.log(self.S0/self.K) +
              ((self.rf-self.div) -
               (self.sigma**2)/2.) *
              self.T) / (self.sigma * math.sqrt(self.T))
        pp_2_inversion = \
            lambda z, n: \
            .5 + math.copysign(1, z) * \
            math.sqrt(.25 - .25 * math.exp(
                -((z/(n+1./3.+.1/(n+1)))**2.)*(n+1./6.)))
        pbar = pp_2_inversion(d1, odd_N)

        self.p = pp_2_inversion(d2, odd_N)
        self.u = 1/self.df * pbar/self.p
        self.d = (1/self.df - self.p*self.u)/(1-self.p)
        self.qu = self.p
        self.qd = 1-self.p

In [11]:
am_option = BinomialLROption(
        50, 50, 0.05, 0.5, 3,
        {"sigma": 0.3, "is_call": False, "is_eu": False})
print("American put: %s" % am_option.price())

American put: 4.8058504579496475


In [12]:
am_option._begin_tree_traversal_()

array([4.80585046])

In [13]:
class BinomialLRWithGreeks(BinomialLROption):

    def _new_stock_price_tree_(self):
        
        self.STs = [np.array([self.S0*self.u/self.d,
                              self.S0,
                              self.S0*self.d/self.u])]

        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches * self.u,
                                 [prev_branches[-1] * self.d]))
            self.STs.append(st)

    def price(self):
        self._setup_parameters_()
        self._new_stock_price_tree_()
        payoffs = self._begin_tree_traversal_()

        """ Option value is now in the middle node at t=0"""
        option_value = payoffs[1]
        
        payoff_up = payoffs[0]
        payoff_down = payoffs[-1]
        S_up = self.STs[0][0]
        S_down = self.STs[0][-1]
        dS_up = S_up - self.S0
        dS_down = self.S0 - S_down

        """ Get delta value """
        dS = S_up - S_down
        dV = payoff_up - payoff_down
        delta = dV/dS

        """ Get gamma value """
        gamma = ((payoff_up-option_value)/dS_up -
                 (option_value-payoff_down)/dS_down) / \
                ((self.S0+S_up)/2. - (self.S0+S_down)/2.)

        return option_value, delta, gamma

In [14]:
eu_call = BinomialLRWithGreeks(50, 50, 0.05, 0.5, 300, {"sigma": 0.3, "is_call": True})
results = eu_call.price()
print("European call values")
print("Price: %s\nDelta: %s\nGamma: %s" % results)

European call values
Price: 3.6605532241706844
Delta: 0.49548913952865475
Gamma: 0.0376617814012847


In [15]:
eu_put = BinomialLRWithGreeks(
        50, 50, 0.05, 0.5, 300, {"sigma":0.3, "is_call": False})
results = eu_put.price()
print("European put values")
print("Price: %s\nDelta: %s\nGamma: %s" % results)

European put values
Price: 4.926309250393408
Delta: -0.504510860471344
Gamma: 0.037661781401289056
